In [1]:
# Define environment variables 
%env DEEP_ROOT=/content/drive/My Drive/brAIn-Power
%env DEEP_DATA=/content/drive/My Drive/correct_data

# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory before importing files
import os
deep_root = os.getenv("DEEP_ROOT")
%cd $deep_root

!pip install import-ipynb
import import_ipynb

env: DEEP_ROOT=/content/drive/My Drive/brAIn-Power
env: DEEP_DATA=/content/drive/My Drive/correct_data
Mounted at /content/drive
/content/drive/My Drive/brAIn-Power
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=5c6d579c817a7bb7051f2a6366b3f2b8b633cda05a104db328cd99b88a1e3d01
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [2]:
from LoadData import CustomDataset, load_data_2np,one_hot_encode,one_hot2_2d
import UNet
import torch
import time
import numpy as np
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
from IPython.display import clear_output

import sklearn.metrics

importing Jupyter notebook from LoadData.ipynb
importing Jupyter notebook from UNet.ipynb


In [14]:
# Define some helper functions for later:
def IoU_score(target, prediction,threshold=0.9):
  ioulst = []
  correct = []
  for j in range(target.shape[0]):
    tar1 = one_hot2_2d(target[j])[0].flatten()
    pred1 = np.argmax(prediction[j],axis=0).flatten()
    iou = sklearn.metrics.jaccard_score(tar1,pred1,average='weighted')
    if iou>=threshold:
      correct.append(1)
    else:
      correct.append(0)

    ioulst.append(iou)
  return np.mean(ioulst),sum(correct)

def Dicecoeff(prediction,label):
    dice = 0
    for i in range(prediction.shape[0]):
      pred1 = one_hot_encode(np.argmax(prediction[i],axis=0))
      intersection = np.sum((pred1 * label[i]))
      dice += (2*intersection+1)/(pred1.sum()+label[i].sum()+1)
    
    return dice/prediction.shape[0]

In [4]:
Valid_perc = 0.15

X_train = np.load(f"data/compressed_data/with_background/X_train,.npy")
Y_train = np.load(f"data/compressed_data/with_background/Y_train,.npy")
X_CAD = np.load(f"data/compressed_data/with_background/X_cad_rnd.npy")
Y_CAD = np.load(f"data/compressed_data/with_background/Y_cad_rnd.npy")
X_test = np.load(f"data/compressed_data/with_background/X_test,.npy")
Y_test = np.load(f"data/compressed_data/with_background/Y_test,.npy")


# Y_train = np.concatenate((Y_train,Y_trans[0:100]),axis=0)
# X_train = np.concatenate((X_train,X_trans[0:100]),axis=0)
Y_train = one_hot_encode(imgs=Y_train)
Y_test = one_hot_encode(imgs=Y_test)
# Y_CAD = one_hot_encode(imgs=Y_CAD)

idx = int(Valid_perc*X_train.shape[0])
X_valid = X_train[0:idx]
X_train = X_train[idx::]
X_train = np.concatenate((X_train,X_CAD),axis=0)

Y_valid = Y_train[0:idx]
Y_train = Y_train[idx::]
Y_train = np.concatenate((Y_train,Y_CAD),axis=0)

In [5]:
DS_train = CustomDataset(X_train, Y_train,one_hot=True)
DS_valid = CustomDataset(X_valid, Y_valid, one_hot=True)
DS_test = CustomDataset(X_test, Y_test, one_hot=True)

In [6]:
# Clear space
del X_train
del Y_train
del X_valid
del X_test
del Y_test

In [7]:
# # Perform preprocessing
DS_train.gray_gamma_enhanced()
DS_valid.gray_gamma_enhanced()
DS_test.gray_gamma_enhanced()

In [8]:
# Check if GPU is available and functions to convert Tensors to Cuda
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")

Running GPU.


In [9]:
# Converts tensors to cuda, if available
def get_variable(x):
    if use_cuda:
        return x.cuda()
    return x

In [10]:
# Get numpy array for both cuda and not
def get_numpy(x):
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [11]:
# Load in model
%%capture
net = torch.load('poster_network.pickle')
net = net.to(torch.float)
if use_cuda:
    print('converting network to cuda-enabled')
    net.cuda()

In [15]:
DL_try = DataLoader(DS_valid)

dice = []
# trained_model.cuda()
for i,data in enumerate(DL_try):
  input = data[0].type(torch.FloatTensor)
  target = data[1].type(torch.FloatTensor)
  
  # Tranfer to GPU if possible
  input = get_variable(input)
  target = get_variable(target)
  
  output = net(input)
  dice.append(Dicecoeff(get_numpy(output),get_numpy(target)))

print('average Dice score:',np.mean(dice))


check  8
check  8
check  8
check  8
check  8
check  8
check  8
check  8
check  7
check  8
check  8
check  8
check  8
check  8
check  8
check  7
check  8
check  8
check  7
check  8
0.9025932114165389


In [ ]:
# Mask should not be one hot encoded (2d)
def plotMaskOnImage(img, mask, remove: int):

    # Mask out 0's
    mask = np.ma.masked_where(mask == 0, mask)

    fig, ax = plt.subplots()
    if len(img.shape)>2:
      ax.imshow(np.moveaxis(img, [0], [-1]))
    else:
      pass
      ax.imshow(img,'gray')
    ax.imshow(np.squeeze(mask),'gist_ncar')
    plt.show()

In [ ]:
val_img = 3
print(DS_valid.labels[val_img].shape,one_hot2_2d(DS_valid.labels[val_img])[0].shape,DS_valid.data[val_img].shape)
# Target 
plotMaskOnImage(DS_valid.data[val_img,0],one_hot2_2d(DS_valid.labels[val_img])[0],0)

# Network output
output = np.squeeze(
            get_numpy(
                net(
                    get_variable(
                        torch.from_numpy(
                            DS_valid.data[val_img][np.newaxis, :, ::]
                            ).type(torch.cuda.FloatTensor)
                        )
                    )
                )
            )

output_2d = np.zeros((256, 256))

mistake_arr = np.zeros((256,256))
# for i in range(256):  # row
#   for j in range(256):  # col
#     if output_2d[i,j] != one_hot2_2d(DS_valid.labels[val_img])[0,i,j]:
#       print(output_2d[i,j],one_hot2_2d(DS_valid.labels[val_img])[0,i,j],i,j)
#       mistake_arr[i,j] = 1
output_2d = np.argmax(output,axis=0)
plt.subplot(221)
plt.imshow(output_2d)

plt.subplot(222)
plt.imshow(one_hot2_2d(DS_valid.labels[val_img])[0])

plt.subplot(223)
plt.imshow(output_2d-one_hot2_2d(DS_valid.labels[val_img])[0])
plt.show()

print(np.amax(output_2d),np.amax(one_hot2_2d(DS_valid.labels[val_img])[0]))
plotMaskOnImage(
    DS_valid.data[val_img,0],
    output_2d,0)